In [23]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib


In [21]:
# Load dataset (replace 'data.csv' with your dataset)
data = pd.read_csv('Housing.csv')



data.head()



,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [4]:
data['mainroad'] = encoder.fit_transform(data['mainroad'])
data['guestroom'] = encoder.fit_transform(data['guestroom'])
data['basement'] = encoder.fit_transform(data['basement'])
data['hotwaterheating'] = encoder.fit_transform(data['hotwaterheating'])
data['airconditioning'] = encoder.fit_transform(data['airconditioning'])
data['prefarea'] = encoder.fit_transform(data['prefarea'])
data['furnishingstatus'] = encoder.fit_transform(data['furnishingstatus'])

In [5]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0


In [6]:
# Define features and target
X = data.drop('price', axis=1)
y = data['price']

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape

(436, 12)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD

# Define the model
model = Sequential()

# Input layer
model.add(Input(shape=(X_train.shape[1],)))

# Additional hidden layer with 15 neurons
model.add(Dense(15, activation='tanh'))

# Hidden layers
model.add(Dense(15, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(15, activation='tanh'))

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer=SGD(), loss='mean_squared_error')


In [14]:
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2)


Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 814657139060297277093445632.0000 - val_loss: 1189158259842204404023296.0000
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 972582531662560503005184.0000 - val_loss: 488862713132117747826688.0000
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 399828421996448028033024.0000 - val_loss: 200971367960354454765568.0000
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 164369641011194921746432.0000 - val_loss: 82619273754450637881344.0000
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 67572247499847172620288.0000 - val_loss: 33964750021943982292992.0000
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27778909566762716495872.0000 - val_loss: 13962889353005354713088.0000
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11419908179512560451584.0000 - val_loss: 5740135525609057026048.0000
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4694729862281309454336.000

In [24]:
# Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate Mean Absolute Error
train_mae = mean_squared_error(y_train, y_train_pred)
test_mae = mean_squared_error(y_test, y_test_pred)

print(f'Training MSE: {train_mse}')
print(f'Testing MSE: {test_mse}')


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Training MAE: 3084039487488.0
Testing MAE: 5130508304384.0


In [18]:
model.save('housing_price_model.h5')


In [16]:
data.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [19]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('housing_price_model.h5')


# Prepare new data
new_data = pd.DataFrame([{
    # Add feature values here
    'area': 1500,
    'bedrooms': 3,
    'bathrooms': 2,
    'stories': 2,
    'mainroad': 1,
    'guestroom': 1,
    'basement': 1,
    'hotwaterheating': 1,
    'airconditioning': 1,
    'parking': 2,
    'prefarea': 1,
    'furnishingstatus':1

}])

# Standardize the new data
new_data_scaled = scaler.transform(new_data)

# Predict using the loaded model
prediction = loaded_model.predict(new_data_scaled)
print(f'Predicted Housing Price: {prediction[0][0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Housing Price: 4731969.0


In [25]:
prediction

array([[4731969.]], dtype=float32)